In [1]:
import importlib as imp
import sys
import numpy as np
sys.path = list(set(sys.path + ['../../']))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import tensorflow as tf

In [3]:
# Constants
NUM_TOTAL_ITEMS = 69669

In [4]:
# Create Metric
import recsys.metrics.auc as auc
auc = imp.reload(auc)
auc_evaluator = auc.AUC()

In [5]:
# Load Data
data_all = np.loadtxt('../data/train_data.dat', delimiter='\t',
                dtype={'names': ('user_id', 'item_id', 'label'), 'formats': ('<i8', '<i8', '<i8')})

In [6]:
# Split Train/Test
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_all)
print(len(data_train), len(data_test))

7875325 2625109


In [7]:
# Create Dataset
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(data_train, total_users=None, total_items=NUM_TOTAL_ITEMS,
                                implicit_negative=False, name='Train')
test_dataset = dataset.Dataset(data_test, total_users=None, total_items=NUM_TOTAL_ITEMS,
                               implicit_negative=False, name='Test')

In [8]:
# Create Featurizer
import homefeeds_featurizer
homefeeds_featurizer = imp.reload(homefeeds_featurizer)
featurizer = homefeeds_featurizer.HomeFeedDataSet("../data/data_formal/", default_item_len=20,
                             user_info_sub_dir ="user_info/", item_info_sub_dir = "item_info/")
#featurizer = homefeeds_featurizer.HomeFeedDataSet("../data/data_formal/", default_item_len=20,
#                             user_info_sub_dir ="user_info_test/", item_info_sub_dir = "item_info/")

init all feature dict begin:
loading: ../data/data_formal/user_info/user_profile_new.res
loading: ../data/data_formal/user_info/user_satisfied_his_json.txt
loading: ../data/data_formal/item_info/item_id_meta_info.txt
loading: ../data/data_formal/item_info/home_feed_reaction_stat.txt
loading: ../data/data_formal/item_info/item_play_info.txt
loading: ../data/data_formal/user_info/user_profile_new.res ../data/data_formal/user_info/user_short_video_info.txt
init all feature dict done.
init all index map begin:
init all index map done.


In [ ]:
# Create Sampler
# create data sampler for training and testing
import recsys.samplers.feeds_sampler as feeds_sampler
feeds_sampler = imp.reload(feeds_sampler)
train_sampler = feeds_sampler.create_training_sampler(
    dataset=train_dataset, featurizer=featurizer, max_pos_neg_per_user=5, batch_size=10, num_process=1, seed=100)
test_sampler = feeds_sampler.create_evaluation_sampler(
    dataset=test_dataset, featurizer=featurizer, max_pos_neg_per_user=30, seed=10)
print("test users: {}".format(len(test_dataset.warm_users(30))))

test users: 280


In [ ]:
fea_user_demography_dim = featurizer.fea_user_demography_dim()
fea_user_stat_dim = featurizer.fea_user_stat_dim()
fea_user_history_dim = featurizer.fea_user_history_dim()
fea_item_meta_dim = featurizer.fea_item_meta_dim()
fea_item_stat_dim = featurizer.fea_item_stat_dim()
fea_context_hour_dim = featurizer.fea_context_hour_dim()
print("fea_user_demography_dim: {}".format(fea_user_demography_dim))
print("fea_user_stat_dim: {}".format(fea_user_stat_dim))
print("fea_user_history_dim: {}".format(fea_user_history_dim))
print("fea_item_meta_dim: {}".format(fea_item_meta_dim))
print("fea_item_stat_dim: {}".format(fea_item_stat_dim))
print("fea_context_hour_dim: {}".format(fea_context_hour_dim))

fea_user_demography_dim: 123
fea_user_stat_dim: 1944
fea_user_history_dim: 20
fea_item_meta_dim: 59226
fea_item_stat_dim: 5
fea_context_hour_dim: 1


In [ ]:
# Create Model
import recsys.recommenders.feeds_fm_recommender as feeds_fm_recommender
feeds_fm_recommender = imp.reload(feeds_fm_recommender)
model = feeds_fm_recommender.FeedsFMRecommender(
    fea_user_demography_dim=fea_user_demography_dim, fea_user_stat_dim=fea_user_stat_dim, fea_user_history_dim=fea_user_history_dim,
    fea_item_meta_dim=fea_item_meta_dim, fea_item_stat_dim=fea_item_stat_dim, fea_context_hour_dim=fea_context_hour_dim,
    total_item_num=NUM_TOTAL_ITEMS)

In [ ]:
# Create Trainer
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
# Start Training
import datetime
print(datetime.datetime.now())

2019-08-08 18:01:17.731263


In [ ]:
# Train
trainer.train(total_iter=100, 
                    eval_iter=5,
                    save_iter=5,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator])

[Training starts, total_iter: 100, eval_iter: 5, save_iter: 5]
[iter 5] Model saved.
[iter 5] loss: nan
..(dataset: Test) evaluation
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt


/home/work/anaconda3/envs/tf13/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/work/anaconda3/envs/tf13/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


..(dataset: Test) AUC nan
[iter 10] Model saved.
[iter 10] loss: nan
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt
..(dataset: Test) AUC nan
[iter 15] Model saved.
[iter 15] loss: nan
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt
..(dataset: Test) AUC nan
[iter 20] Model saved.
[iter 20] loss: nan
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt
..(dataset: Test) AUC nan
[iter 25] Model saved.
[iter 25] loss: nan
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt
..(dataset: Test) AUC nan
[iter 30] Model saved.
[iter 30] loss: nan
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt
..(dataset: Test) AUC nan
[iter 35] Model saved.
[iter 35] loss: nan
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FeedsFMRec/model.ckpt
..(dataset: Test) AUC nan
[i

In [ ]:
# Finish Training
print(datetime.datetime.now())

In [ ]:
# Export Protobuf model for online serving
!rm -rf ./pbModel
model.export(export_model_dir="pbModel", as_text=False)

In [ ]:
# Inspect internal graph weights
def inspect_weights(checkpoint_model_dir):
    reader = tf.train.NewCheckpointReader(checkpoint_model_dir)
    variables = reader.get_variable_to_shape_map()
    for var, shape in variables.items():
        trained_weight = reader.get_tensor(var)
        print("VarName: {}".format(var))
        print("VarShape: {}".format(shape))
        print("VarValue: {}".format(trained_weight))
inspect_weights("./FeedsFMRec/model.ckpt")

In [ ]:
# Inspect internal op tensor value of testing
b = test_sampler.next_batch()
print("data:", b)
print("score:", model.serve_inspect_ports(batch_data=b[1], ports=model.servegraph.get_outputs()))